For the informations that are not about people (and likely not on Wikidata) I'll try a different strategy. 


For each section of each page, I'll get the Wikitext. 

From the wikitext, I'll extract, for each entry in a section:
    
    - the date string
    - the full wikitext of the section

Then I'll build functions to:
    - Parse date string in two (if possible)
    - Get the first link of the entry
    - Get the additional link URLs of the entry
    - Convert wikitext into readable human text.
    
   

In [2]:
import requests
from functions import functions
import pandas as pd
query = "https://en.wikipedia.org/w/api.php?action=parse&prop=sections&page=17th_century&format=json"
sections = requests.get(query)
sections_df = pd.json_normalize(sections.json()["parse"]["sections"])
sections_df.head()

,toclevel,level,line,number,index,fromtitle,byteoffset,anchor
0,1,2,Events,1,1,17th_century,7373,Events
1,2,3,1601–1650,1.1,2,17th_century,7423,1601–1650
2,2,3,1651–1700,1.2,3,17th_century,17327,1651–1700
3,1,2,Significant people,2,4,17th_century,23000,Significant_people
4,2,3,Musicians,2.1,5,17th_century,27283,Musicians


I'll start the exploration with the first section: Events

In [4]:

page = "17th_century"
section = "1"
query = "https://en.wikipedia.org/w/api.php?action=parse&format=json&page=" + page + "&prop=wikitext&section=" + section + "&disabletoc=1"
wikitext = requests.get(query)


In [12]:
section_text = wikitext.json()["parse"]["wikitext"]["*"]

In [16]:
section_text_in_chunks = section_text.split("*")

In [36]:
df = pd.DataFrame(columns=["date_string", "wikitext_string"])

for text in section_text_in_chunks:
    data = text.split(":",1)
    
    if len(data) == 2:
        row = {"date_string":data[0], "wikitext_string":data[1]}
        df = df.append(row, ignore_index=True)


In [40]:
df.head()

,date_string,wikitext_string
0,[[1600]],[[Michael the Brave]] unifies the three [[Rom...
1,[[1601]],"[[Battle of Kinsale]], England defeats Irish ..."
2,[[1601]]–[[1603]],The [[Russian famine of 1601–1603]] kills per...
3,[[1602]],[[Matteo Ricci]] produces the [[Kunyu Wanguo ...
4,[[1602]],The [[Dutch East India Company]] (VOC) is est...


Nice, now we have the raw material to build further. We will use a Python library for wikitext to extract the informations of interest. This is the library that will be used:
https://pypi.org/project/wikitextparser/#id11